<a href="https://colab.research.google.com/github/sim-88/Deep-Learning-projects/blob/main/CommentToxicity2104193.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#!pip install tensorflow-gpu==2.12.0

##  0. **Bring In Data**

In [30]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [31]:

df=pd.read_csv("/content/drive/MyDrive/cmmntToxic/train.csv")

In [32]:
df[df['toxic']==1].head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0


In [33]:
df.iloc[7]['comment_text']

"Your vandalism to the Matt Shirvington article has been reverted.  Please don't do it again, or you will be banned."

In [34]:
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

##  **1. Preprocess**

In [35]:
from keras.layers import TextVectorization

In [36]:
X=df['comment_text']
y=df[df.columns[2:]].values

In [37]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [38]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [39]:
MAX_FEATURES=200000    #Number of words in the vocab

In [40]:
vectorizer=TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')

In [41]:
vectorizer.adapt(X.values)

In [42]:
vectorizer("Hello world,life is great")

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([288,   1,   9, ...,   0,   0,   0])>

In [43]:
vectorized_text=vectorizer(X.values)
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [44]:
len(X)

159571

In [45]:
#TENSORFLOW DATA PIPELINE
#MACSHBAP - map,cache, shuffle, batch, prefetch
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [46]:
batch_X,batch_y =dataset.as_numpy_iterator().next()

In [47]:
batch_X.shape

(16, 1800)

In [48]:
batch_y.shape

(16, 6)

In [28]:
train = dataset.take(int(len(dataset) * .7))
val = dataset.skip(int(len(dataset) * .7)).take(int(len(dataset) * .2))
test = dataset.skip(int(len(dataset) * .9)).take(int(len(dataset) * .1))

## 2.Create Sequential Model

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding

In [50]:
model=Sequential()
#Create the embedding layer
model.add(Embedding(MAX_FEATURES+1,32))
#Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32,activation='tanh')))
#Feature extractor Fully Connected Layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#Final layer
model.add(Dense(6,activation='sigmoid'))

In [51]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [53]:
fitt=model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 16223s 2s/step - loss: 0.0625 - val_loss: 0.0477


In [54]:
fitt.history

{'loss': [0.06254575401544571], 'val_loss': [0.04771500453352928]}

In [56]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(8,5))
# pd.DataFrame(fitt.history).plot()
# plt.show()

## **3. Make** **Predictions**

In [57]:
input_text=vectorizer("You freaking suck! I am going to hit you.")

In [58]:
batch=test.as_numpy_iterator().next()

In [59]:
batch_X, batch_y=test.as_numpy_iterator().next()

In [60]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [61]:
(model.predict(batch_X)>0.5).astype(int)

1/1 [==============================] - 1s 1s/step


array([[0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [62]:
res=model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 170ms/step


##   **4. Evaluate Model**

In [63]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [64]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [65]:
for batch in test.as_numpy_iterator():
  #unpack the batch
  X_true,y_true=batch
  #Make a prediction
  yhat=model.predict(X_true)

  #Flatten the predictions
  y_true=y_true.flatten()
  yhat=yhat.flatten()

  pre.update_state(y_true,yhat)
  re.update_state(y_true,yhat)
  acc.update_state(y_true,yhat)

1/1 [==============================] - 0s 248ms/step


In [67]:
print(f"Precision:{pre.result().numpy()},Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}")

Precision:0.8346893787384033,Recall:0.6593631505966187,Accuracy:0.49949848651885986


## **5. Test And Gradio**

In [68]:
!pip install gradio jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [69]:
import gradio as gr

In [70]:
model.save("CommentToxicity.h5")

In [71]:
model=tf.keras.models.load_model('CommentToxicity.h5')

In [79]:
input_str=vectorizer("hey i freaken hate you!I'm coming to hunt you down.I'm going to hurt you!")

In [80]:
res=model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 0s 156ms/step


In [75]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [81]:
res

array([[0.5259967 , 0.01300728, 0.1770336 , 0.02331325, 0.2170917 ,
        0.06248635]], dtype=float32)

In [97]:
def score_comment(comment):
  vectorized_comment=vectorizer(comment)
  results=model.predict(np.expand_dims(vectorized_comment,0))

  text=''
  for idx,col in enumerate(df.columns[2:]):
    text+='{}: {}\n'.format(col,results[0][idx]>0.5)

  return text

In [101]:
# interface=gr.Interface(fn=score_comment,inputs=gr.inputs.Textbox(lines=2,placeholder="Comment to score"),outputs='text')
iface = gr.Interface(
    fn=score_comment,
    inputs=Textbox(lines=2, placeholder="Comment to score"),
    outputs=Text()
)

In [ ]:
iface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e8c426b12f90cafdd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 331ms/step
